In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import numpy as np
import time
import pandas as pd


# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from random import choices

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


print(tf.executing_eagerly())
print(tf.__version__)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

print('GPUs available: {}'.format(len(physical_devices)))

True
2.2.0
GPUs available: 1


In [2]:
def evaluate_model(model_dir, data, labels, kfolds=5, scale_images = False, seed=0):
    model = tf.keras.models.load_model(model_dir)

    #Forward pass to retrieve scene embeddings
    scale_images = False

    scene_embeddings = []

    mean = data.mean(axis=0)
    std = np.std(data, axis=0)

    data_scaled = (data - mean)/std

    for i in range(num_images):
        if scale_images:
            scene_embedding = np.array(model(data_scaled[i:i+1]))
        else:
            scene_embedding = np.array(model(data[i:i+1]))

        scene_embeddings.append(scene_embedding.flatten())

    scene_embeddings = np.stack(scene_embeddings)
    
    skf = StratifiedKFold(n_splits=kfolds, random_state = seed)
    
    svm_params = {'kernel':'linear', 'random_state':seed, 'C':1}
    clf = make_pipeline(StandardScaler(), SVC(**svm_params))
    
    accuracy = []
    accuracy_min = []
    accuracy_max = []
    f1_micro = []
    f1_min = []
    f1_max = []
    f1_weighted = []

    
    for train_index, test_index in skf.split(scene_embeddings, labels):
        X_train, X_test = scene_embeddings[train_index], scene_embeddings[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        
        clf.fit(X_train, y_train)
        
        predictions = clf.predict(X_test)
        
        f1 = f1_score(y_test, predictions, average=None)
        f1_min.append(np.min(f1))
        f1_max.append(np.max(f1))
        f1_weighted.append(f1_score(y_test, predictions, average='weighted'))
        f1_micro.append(f1_score(y_test, predictions, average='micro'))
        
        cm = confusion_matrix(y_test, predictions)
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        accuracies = cm.diagonal()
        accuracy.append(np.mean(accuracies))
        accuracy_max.append(np.max(accuracies))
        accuracy_min.append(np.min(accuracies))
        
    accuracy_std = np.std(accuracy)
    accuracy = np.mean(accuracy)
    
    accuracy_min = np.mean(accuracy_min)
    accuracy_min_min = np.min(accuracy_min)
    accuracy_max = np.mean(accuracy_max)
    accuracy_max_max = np.max(accuracy_max)
    
    f1_micro_std = np.std(f1_micro)
    
    f1_min = np.mean(f1)
    f1_min_min = np.min(f1)
    
    f1_max = np.mean(f1_max)
    f1_max_max = np.max(f1_max)
    
    f1_weighted = np.mean(f1_weighted)

    
    return {model_dir: [accuracy, accuracy_std, accuracy_min, accuracy_min_min, accuracy_max, accuracy_max_max, f1_micro_std, f1_weighted, f1_min, f1_min_min, f1_max, f1_max_max]}

#### Import data

In [3]:
data_dir = "data/UCM.tfrecords"

# Define the features in the TFRecords file
features = {
    "image_raw": tf.io.FixedLenSequenceFeature(
        [], dtype=tf.float32, allow_missing=True
    ),
    "height": tf.io.FixedLenFeature([], tf.int64),
    "width": tf.io.FixedLenFeature([], tf.int64),
    "channels": tf.io.FixedLenFeature([], tf.int64),
    "label": tf.io.FixedLenFeature([], tf.int64),
}

def parse_example(example_proto):
    image_features = tf.io.parse_single_example(example_proto, features)
    
    height = tf.cast(image_features["height"], tf.int32)
    width = tf.cast(image_features["width"], tf.int32)
    channels = tf.cast(image_features["channels"], tf.int32)
    
    image_raw = tf.reshape(
        tf.squeeze(image_features["image_raw"]),
        tf.stack([height, width, channels])
    )
    
    target = image_features["label"]
    
    return image_raw, target

with open('data/label_dict.pkl', 'rb') as f:
    label_dict = pkl.load(f)
    
tfdata = tf.data.TFRecordDataset(data_dir).map(parse_example, num_parallel_calls=tf.data.experimental.AUTOTUNE)

data_numpy = np.stack([x[0] for x in list(tfdata.as_numpy_iterator())]) #Could change this to not load whole dataset into memory if too big
labels = np.stack([x[1] for x in list(tfdata.as_numpy_iterator())])

num_images = data_numpy.shape[0]
image_size = data_numpy.shape[1]
no_channels = data_numpy.shape[3]

In [4]:
%%time
results_dict = {}

for model_name in filter(lambda s:s.startswith('UnsupervisedFeatureLearning'), os.listdir('.')):
    model_results = evaluate_model(model_name, data_numpy, labels, seed=1)
    results_dict = {**results_dict, **model_results}
    
df = pd.DataFrame.from_dict(results_dict, orient='index', columns=['accuracy', 'accuracy_std', 'accuracy_min (mean)', 'accuracy_min (min)', 'accuracy_max (mean)', 'accuracy_max (max)', 'f1_micro_std', 'f1_weighted', 'f1_min (mean)', 'f1_min (min)', 'f1_max (mean)', 'f1_max (max)'])
df.to_excel('Results_{}.xlsx'.format(time.strftime("%Y%m%d-%H%M%S")))

/home/matt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


/home/matt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


KeyboardInterrupt: 